In [20]:
import math 
import string 
import re

import json 

import pandas as pd 
import sqlite3



create_table_generated_summaries_str =  '''CREATE TABLE generated_summaries (
    uuid INTEGER PRIMARY KEY AUTOINCREMENT, 
    docid TEXT NOT NULL ,
    model TEXT NOT NULL, 
    summary TEXT NOT NULL, 
    dialogue TEXT
);'''

create_table_label_str = '''CREATE TABLE label (
    uuid INTEGER PRIMARY KEY AUTOINCREMENT, 
    user_id TEXT NOT NULL,
    docid TEXT NOT NULL,
    model TEXT NOT NULL,
    nonfactual_spans ENUM ,
    evidence TEXT ,
    error_type TEXT,
    summary TEXT NOT NULL,
    dialogue TEXT
);'''




 
# filter_ids = ["'" + each + "'" for each in filter_ids]
# str_filter_ids = '(' + ', '.join(filter_ids) + ')'

def connect_to_db(db_path):
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    return conn, c 


def create_table(create_str, db_path):
    conn, c = connect_to_db(db_path)
    c.execute('''%s'''%(create_str))
    conn.commit()
    conn.close()

def add_data(filename, db_path):
    df = pd.read_csv(filename)
    print(len(df))
    conn, c = connect_to_db(db_path)
#     print('Size before adding' , len(c.execute("""SELECT * from generated_summaries""").fetchall()))
    for idx, row in df.iterrows():
        # print(row)
        docid = row['DocID']
        dialogue = row['Dialogue']
        model = row['Model']
        origin = row['origin']
        summary = row['Summary']
        
        if 'Summary' in df.keys() and row['DocID'] in filter_ids:
            generic_summary = row['Summary']
            summary_uuid_generic = f'{docid}_{model}_gen'
            c.execute("""INSERT INTO generated_summaries (docid, model, summary, dialogue) VALUES (?, ?, ?, ?)""",
                                                        (docid, 
                                                        model,
                                                        summary,
                                                        dialogue))
            
            
    
    print('Size after adding' , len(c.execute("""SELECT * from generated_summaries""").fetchall()))
    conn.commit()
    conn.close()
    
def create_tables(db_path, force_new):
    print(force_new)
    conn, c = connect_to_db(db_path)
    create_cmds = {
        'generated_summaries': create_table_generated_summaries_str,
        'label': create_table_label_str
    }
    
    for table_name, create_str in create_cmds.items():
        table_results = c.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}';").fetchall()
        print(table_results)
        if len(table_results) != 0:
            print('Size before adding' , len(c.execute("""SELECT * from generated_summaries""").fetchall()))
            if (force_new == True):
                print('here', force_new)
                print(f'Dropping {table_name} ...')
                c.execute(f'DROP table {table_name};')
                create_table(create_str, db_path)
        else:
             create_table(create_str, db_path)

In [21]:
len(filter_ids)

83

In [6]:
database_path = '/home/sanjana/explainable_factual_evaluation/datasets/short_dialogue/model_generated/annotations/gpt4_summaries_short.db'


In [7]:
# path = '/home/sanjana/dialogue-summarization-annot/data/gpt4_summaries_short_qualifying.db'
conn, c = connect_to_db(database_path)
# conn, c = connect_to_db('/home/sanjana/explainable_factual_evaluation/datasets/short_dialogue/model_generated/annotations/gpt4_summaries_short.db')
# df_path = '/home/sanjana/explainable_factual_evaluation/datasets/short_dialogue/model_generated/gpt4_generated_summaries/RefMatters_DialogSum_test.csv'
# create_tables(database_path, force_new = True)
# add_data(df_path, database_path)
# pd.read_csv(df_path)

In [16]:
c.execute(f"""SELECT error_type, nonfactual_spans, summary FROM label WHERE user_id == 'elisa'""").fetchall()
   
# nonfactual_spans, evidence, summary

[('Intrinsic_Error',
  'Person 1 (Steven) noticed that Pe',
  "Person 1 (Steven) noticed that Person 2 (Tony) looked unhappy. Tony admitted making a big mistake, revealing he got caught cheating on a test which made him feel ashamed. He explains his fear of failing due to his father's expectations. However, Steven advises him to study hard and learn from his mistakes."),
 (None,
  '',
  'Person1 checks if Person2 has fed the puppies, which they have. Person1 then instructs Person2 to bathe the puppies later and reminds them about a vet appointment on Saturday. Person2 confirms they will do both and asks for the timing of the vet appointment, which is scheduled for eleven in the morning.'),
 ('Intrinsic_Error',
  'Person1 inquires about the me',
  "Person1 inquires about the method to contact Person2, who suggests either phone contact or email. Person2 further asks when Person1's organization will make a decision, to which Person1 responds it will take approximately one week. Person2 sh

In [12]:
conn.commit()

In [60]:
str_filter_ids

'(test_12, test_120, test_123, test_127, test_127, test_132, test_133, test_140, test_140, test_147, test_149, test_149, test_149, test_149, test_161, test_17, test_161, test_18, test_182, test_182, test_191, test_194, test_194, test_201, test_206, test_208, test_21, test_210, test_212, test_219, test_22, test_228, test_236, test_239, test_241, test_245, test_246, test_261, test_264, test_266, test_267, test_269, test_27, test_282, test_284, test_287, test_29, test_299, test_300, test_306, test_309, test_310, test_311, test_312, test_315, test_317, test_318, test_320, test_323, test_327, test_328, test_33, test_336, test_337, test_338, test_339, test_341, test_343, test_346, test_35, test_350, test_352, test_353, test_354, test_360, test_363, test_363, test_364, test_367, test_371, test_372, test_373, test_376)'

In [47]:
from typing import Dict, Tuple, Sequence


import sqlalchemy
from sqlalchemy import text, create_engine, Index, MetaData, Table, select, exists
# from sqlalchemy.sql import and_

from random import shuffle
# from werkzeug.security import generate_password_hash, check_password_hash
import math
import spacy


directory = '/home/sanjana/explainable_factual_evaluation/datasets/short_dialogue/model_generated/annotations'
database_name = 'gpt4_summaries_short.db'
db_path = '/%s/%s'%(directory, database_name)
db_engine = dbEngine=sqlalchemy.create_engine('sqlite:///' + db_path)

metadata = MetaData(bind=db_engine)
metadata.reflect()
print(metadata)
generated_summaries = metadata.tables['generated_summaries']
label = metadata.tables['label']

MetaData(bind=Engine(sqlite://///home/sanjana/explainable_factual_evaluation/datasets/short_dialogue/model_generated/annotations/gpt4_summaries_short.db))


In [48]:
subquery_1 = select([label.c.error_type, ]).where(label.c.user_id == 'elisa')
subquery_2 = select(generated_summaries.c.docid)
# .where(generated_summaries.c.docid == label.c.docid)
        
with dbEngine.connect() as con:
            #stmt = select(generated_summaries.c.summary_uuid).limit(1)
            # ans1 = con.execute(subquery_1).fetchall()
            # ans2 = con.execute(subquery_2).fetchall()
            results = list(con.execute(subquery_1).fetchall())
            # results = [each[0] for each in results]
            print('Subquery', len(results))
            # print('Subquery', con.execute(subquery_2).fetchone()[0])
            # print(con.execute(subquery_2).fetchall())
            # print('Subquery', ans2[0][0] )

Subquery 83


In [50]:
len(results)

83